In [4]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta
import altair as alt
from pandas.io.json import json_normalize
import requests
import yahoo_fin.stock_info as yf
from alpha_vantage.fundamentaldata import FundamentalData
import json
from fbprophet import Prophet 

D = 400
date_D_days_ago = datetime.now() - timedelta(days=D)
now = datetime.now()

start_date = date_D_days_ago.strftime('%F')
end_date = now.strftime('%F')

apikey='2HCDHLMKIK99309W'
tech_stocks = ['AAPL', 'PTON', 'TSLA']


tech_stocks[0]


# Operating CashFlow minus Capital Expenditures


'AAPL'

In [5]:
#Get Data from Yahoo
def StockData(ticker, start_d, end_d):
    data = web.get_data_yahoo(ticker, start = start_d, end = end_d)
    price =  pd.DataFrame(data['Adj Close'])
    
    #volume = pd.DataFrame(data['Volume'])
    return price

In [6]:
price = StockData(tech_stocks,start_date,end_date)

In [ ]:
test = price[tech_stocks[0]].reset_index()

test.rename(columns ={'Date':'ds','AAPL':'y'}, inplace=True)

In [ ]:
prophet = Prophet()
prophet.fit(test)

In [ ]:
future_prices=prophet.make_future_dataframe(periods=365)
future = prophet.predict(future_prices)

In [ ]:
f = future[['ds','trend','yhat_lower','yhat_upper']]

In [ ]:
f

In [ ]:

f = f.melt('ds', var_name='bands', value_name='price')
f

In [ ]:
pic = alt.Chart(f).mark_line().encode(
        x='ds:T',
        y='price:Q',
        color='bands:N', tooltip=['bands:N', 'ds:N','price:N']
    )

In [ ]:
pic

In [ ]:
prophet.plot_components(future)

In [ ]:
test.columns = ['a','b']

In [ ]:
test


In [ ]:
df1 = pd.DataFrame({'key' : ['a','b','c'],'value': range(3)}, index = ['a1','x2','a3'])
df2 = pd.DataFrame({'key' : ['x','y','x','v'],'stuff': range(4)}, index = ['a1','a2','a3','a4'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
df1.combine_first(df2)

In [ ]:
df1.index.str.match('a3').any()

In [ ]:
df1.columns.values[0]

In [ ]:
%pip install streamlit-disqus

In [7]:
price

Symbols,AAPL,PTON,TSLA
Date,,,
2020-01-08,75.036385,29.650000,98.428001
2020-01-09,76.630219,27.870001,96.267998
2020-01-10,76.803459,28.010000,95.629997
2020-01-13,78.444321,28.530001,104.972000
2020-01-14,77.385063,29.690001,107.584000
...,...,...,...
2021-02-04,137.184998,157.529999,849.989990
2021-02-05,136.759995,148.300003,852.229980
2021-02-08,136.910004,144.990005,863.419983
